# TF-IDFを試す。

In [9]:
# 前処理用
import re
import os
import pandas as pd
import numpy as np
import emoji
import spacy
import neologdn
import json

# 分かち書き用
import ginza
import ja_ginza_electra
# pandas高速化
from pandarallel import pandarallel
pandarallel.initialize()
import json
import os
import emoji
import mojimoji
import neologdn
from tqdm.auto import tqdm
tqdm.pandas()

print("Load data!")
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Load data!


In [10]:
emoji_json_path = "../emoji/emoji_ja.json"
json_open = open(emoji_json_path)
emoji_dict = json.load(json_open)
nlp = spacy.load('ja_ginza_electra')

def wakati_rm_func(x):
    sentence = str(x)
    sentence = re.sub(r'[!-~]'," ",sentence) # 小文字の記号を削除
    sentence=re.sub(r'[︰-＠]', "", sentence) # 大文字の記号を削除


    # 不要記号削除
    pattern = '[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”◇ᴗ●↓→♪★⊂⊃※△□◎〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％�]'
    sentence =  re.sub(pattern, ' ', sentence)

    # 正規化する
    sentence = neologdn.normalize(sentence)

    # 大文字・小文字変換
    sentence = sentence.replace("\n", "")
    sentence = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+", "", sentence)
    # 絵文字削除
    sentence = "".join(
        [
            "絵文字" + emoji_dict.get(c, {"short_name": ""}).get("short_name", "")
            if c in emoji.UNICODE_EMOJI["en"]
            else c
            for c in sentence
        ]
    )
    # GinZaで分かち書きをする
    doc = nlp(sentence)
    tmp_words_list = []
    for sent in doc.sents:
        for token in sent:
            if token.pos_ in ["PROPN", "NOUN", "ADJ", "VERB"]:
                tmp_words_list.append(token.orth_)

    result = " ".join(tmp_words_list)
    return result

In [14]:
wakati_rm_func(train["story"][0])

'新しく 手直し 宇宙 開拓 地 入植 農家 子供 小人 生まれる 両親 子 受け入れよう し 飼い猫 ロマノフ 世話 し くれる 子供達 受け入れ くれ 子供達 意地悪 より ロマノフ 死ん しまう 両親 家庭 教師 宇宙 開拓 局 役人 クリス 迎える 授業 中 学ぶ 楽しみ 見付け 行く 地球 政府 革命 軍 戦い 為 元 去っ いく'

In [20]:
pandarallel.initialize(progress_bar=True)
df = pd.concat([train, test])
# trainに分かち書きを実行する
df["title_wakati"] = df["title"].progress_apply(wakati_rm_func)
df["story_wakati"] = df["story"].progress_apply(wakati_rm_func)
df["keyword_wakati"] = df["keyword"].progress_apply(wakati_rm_func)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


  0%|          | 0/48522 [00:00<?, ?it/s]

  0%|          | 0/48522 [00:00<?, ?it/s]

  0%|          | 0/48522 [00:00<?, ?it/s]

In [24]:
df

,ncode,general_firstup,title,story,keyword,userid,writer,biggenre,genre,novel_type,...,isbl,isgl,iszankoku,istensei,istenni,pc_or_k,fav_novel_cnt_bin,title_wakati,story_wakati,keyword_wakati
0,N7588B,2007-04-01 16:40:57,暗い窓辺と皇帝の北の宮,※新しく手直ししました。\n\nhttp://ncode.syosetu.com/n2539...,ファンタジー SF 天使 小人 猫,9904,梅田浩志,4,402,1,...,0,0,0,0,0,0,1.0,暗い 窓辺 皇帝 北 宮,新しく 手直し 宇宙 開拓 地 入植 農家 子供 小人 生まれる 両親 子 受け入れよう し...,ファンタジー 天使 小人 猫
1,N8726B,2007-04-22 15:40:30,仮想現実ゲーム 『ヴァルハラ』,ゲームセンターの片隅に置かれた律儀なまでにリアルなゲーム、ヴァルハラ。それはプレイを重ねるた...,仮想現実 ゲーム バトル 戦乙女 高校生 学校/学園 アンドロイド ライト SF エンターテ...,6527,ルト,4,401,1,...,0,0,0,0,0,3,2.0,仮想現実 ゲームヴァルハラ,ゲームセンター 片隅 置か 律儀 リアル ゲームヴァルハラ プレイ 重ねる たび たち 魅了...,仮想現実 ゲーム バトル 戦 乙女 高校生 学校 学園 アンドロイドライトエンターテイメント...
2,N5481C,2007-08-22 18:25:54,海の竜騎兵,海軍の中に存在するクジラ部隊「竜騎兵」 主人公はその一員だ。\n騎乗する鯨との間に結ばれた絆...,ヒーロー 冒険 海軍 深海 海中の戦闘 HJ2021,13882,雨宮雨彦,4,403,1,...,0,0,0,0,0,0,3.0,海 竜騎兵,海軍 中 存在 クジラ 部隊 竜騎兵 主人公 一員 騎乗 鯨 間 結ば 絆 強く 主人公 戦...,ヒーロー 冒険 海軍 深海 海中 戦闘
3,N6260C,2007-09-06 21:33:37,宇宙狂時代,はるか未来、宇宙船メーカー「ペガサス」のわずか十八才の社長でもあり、美少女でもあるミリィは、...,冒険 ロケット ロボット サイボーグ レース コメディ SF エンターテイメント 少女 友情...,13920,万卜人,4,402,1,...,0,0,0,0,0,2,1.0,宇宙 狂 時代,はるか 未来 宇宙船 メーカー ペガサス 才 社長 あり 美少女 ある ミリィ 前代未聞 宇...,冒険 ロケットロボットサイボーグレースコメディエンターテイメント 少女 友情 少年 未来 宇宙
4,N6903C,2007-09-17 01:02:33,クライムズ クライシス,アメリカの大富豪テッドマクシミリアン。心筋梗塞を患った彼があの世に旅立った。弁護士が開いた遺...,ゲーム フェア 平和 権力 ドキドキ シリアス ハッピーエンド 国家/民族 友情 OL/サラ...,12136,武上 渓,4,401,1,...,0,0,0,0,0,1,1.0,クライムズクライシス,アメリカ 大富豪 テッド マクシミリアン 心筋 梗塞 患っ あの世 旅立っ 弁護士 開い 遺...,ゲーム フェア 平和 権力 ドキドキシリアスハッピーエンド 国家 民族 友情 サラリーマン ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8517,N5983HF,2021-09-23 21:23:17,睡眠エネルギー,眠れなくてお悩みの方へ。私は不眠目前の時期もあったけれど、今は比較的気持ちよく寝ています。眠...,日常 睡眠 健康 不眠,2263100,アボカドまぐろ,99,9903,2,...,0,0,0,0,0,2,NaN,睡眠 エネルギー,眠れ お 悩み 方 不眠 目前 時期 あっ 今 比較的 気持ち よく 寝 い 眠れ 方 価値...,日常 睡眠 健康 不眠
8518,N5990HF,2021-09-23 21:30:35,中秋の名月に照らされたオオカミ男,\n犬歯が目立つことで、冷やかされているクラスメイト。 \n「オオカミ男」「オオカミ男」とク...,日常 シリアス 人外 現代 狼男 月 ホラー 切ない ショートショート 短編,2254651,かごめかごめ,3,305,2,...,0,0,0,0,0,2,NaN,中秋 名月 照らさ オオカミ 男,犬歯 目立つ こと 冷やかさ いる クラスメイト オオカミ 男 オオカミ 男 クラス 中 囃...,日常 シリアス 人外 現代 狼男 月 ホラー 切ない ショートショート 短編
8519,N5993HF,2021-09-23 21:34:30,敬老の日、継続×3,敬老の日から三日間が経ちました。男子三日会わざれば刮目して見よといいますが、僕は相変わらずの...,日常 私小説,846989,MiddleGeass,3,302,2,...,0,0,0,0,0,2,NaN,敬老の日 継続,敬老の日 日間 経ち 男子 日 会わ 刮目 見よ いい 相変わらず,日常 私小説
8520,N5996HF,2021-09-23 21:42:25,ひよっこ探偵と天然魔女が出会うまで,とある探偵事務所で働く主人公と世界中の話題となっている魔女と出会うまでの些細な小話。\n\n...,NaN,2259847,猫宮ピグロ,2,202,2,...,0,0,0,0,0,2,NaN,ひよっこ 探偵 天然 魔女 出会う,探偵 事務所 働く 主人公 世界中 話題 なっ いる 魔女 出会う 些細 小話 プロローグ ...,


In [28]:
tmp_df = df[["ncode", "title_wakati", "story_wakati", "keyword_wakati"]]
tmp_train = tmp_df.iloc[:40000]
tmp_test = tmp_df.iloc[40000:]

tmp_train.to_pickle("../data/wakati_train.pkl")
tmp_test.to_pickle("../data/wakati_test.pkl")

In [26]:
train = df.iloc[:40000]
test = df.iloc[40000:]
print(df.shape)

(48522, 23)


In [ ]:
print(train.shape)
print(test.shape)

(40000, 20)
(8522, 19)


In [ ]:
train_["story_wakati"][:5]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
39995   NaN
39996   NaN
39997   NaN
39998   NaN
39999   NaN
Name: story_wakati, Length: 40000, dtype: float64

In [ ]:
# TI-IDFを計算する
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer()
X = model.fit_transform(train["story_wakati"])
story_tfidf_train = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

model = TfidfVectorizer()
X = model.fit_transform(train["title_wakati"])
title_tfidf_train = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

model = TfidfVectorizer()
X = model.fit_transform(train["keyword_wakati"])
keyword_tfidf_train = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

pd.concat([story_tfidf_train, title_tfidf_train, keyword_tfidf_train]).to_pickle("data/train_tfidf.pkl")

,欠損
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
39995,1.0
39996,1.0
39997,1.0
39998,1.0


In [ ]:
model = TfidfVectorizer()
X = model.fit_transform(test["story_wakati"])
story_tfidf_test = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

model = TfidfVectorizer()
X = model.fit_transform(test["title_wakati"])
title_tfidf_test = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())

model = TfidfVectorizer()
X = model.fit_transform(test["keyword_wakati"])
keyword_tfidf_test = pd.DataFrame(data= X.toarray(), columns = model.get_feature_names())
print(story_tfidf_test.shape)
pd.concat([story_tfidf_test, title_tfidf_test, keyword_tfidf_test]).to_pickle("data/test_tfidf.pkl")